In [26]:
data Lens a b = Lens (a -> b) (b -> a -> a)

modify :: Lens a b -> (b -> b) -> a -> a
modify (Lens get set) f a = set (f (get a)) a

set :: Lens a b -> b -> a -> a
set (Lens _ s) b = s b

-- Lenses are composable

andThen :: Lens a b -> Lens b c -> Lens a c
andThen (Lens g1 s1) (Lens g2 s2) = Lens (g2 . g1) (\c a -> s1 (s2 c (g1 a)) a) -- two sets, one get

-- Example lenses

trivial :: Lens a ()
trivial = Lens (const ()) (const id)

self :: Lens a a
self = Lens id (\a _ -> a)

fstl :: Lens (a,b) a
fstl = Lens fst (\a (_,b) -> (a,b))

sndl :: Lens (a,b) b
sndl = Lens snd (\b (a,_) -> (a,b))

foo = (1,(2,3))

set (sndl `andThen` sndl) 5 foo

--

import Prelude hiding (lookup, insert)
import Data.Map.Strict

member :: Ord k => k -> Lens (Map k v) (Maybe v)
member k = Lens (lookup k) set 
    where set (Just v) = insert k v
          set Nothing  = delete k
          
m = fromList [(1,'a'),(2,'b')]

set (member 1) (Just 'c') m

-- State 

import Control.Monad.State

asState :: Lens a b -> State a b
asState (Lens get _) = state (\s -> (get s, s)) 

(<~) :: Lens s a -> a -> State s a
(<~) (Lens _ set) a = state (\s -> (a, set a s))


postInc :: Lens s Int -> State s Int
postInc lens = do
        x <- supply
        lens <~ (x + 1)
        return x
    where supply = asState lens
    
focus :: State s a -> Lens t s -> State t a
focus st (Lens get set) = state st'
    where st' t = let (a,s) = runState st (get t)
                  in  (a, set s t)
                  
data Foo = Foo { _n :: Int }
            deriving Show

n :: Lens Foo Int
n = Lens _n (\m foo -> foo { _n = m })

runState (postInc n) (Foo 1)

data Bar = Bar { _foo :: Foo }
            deriving Show

foo :: Lens Bar Foo
foo = Lens _foo (\foo bar -> bar { _foo = foo })

st = focus (postInc n) foo

runState st (Bar $ Foo 1)

-- modify
(%=) :: Lens s a -> (a -> a) -> State s a
(%=) lens f = do
    x <- asState lens
    y <- lens <~ (f x)
    return y
    
(+=) :: Lens a Int -> Int -> State a Int
(+=) lens n = lens %= (+ n)

st = do
    x <- asState n
    n += 3
    return x
    
runState st (Foo 7)

data Moo = Moo { _t :: (Int, (Int,Int)) }
            deriving Show

t :: Lens Moo (Int,(Int,Int))
t = Lens _t (\t m -> m { _t = t })

st = do t <~ (1,(2,3))
        (t `andThen` sndl `andThen` sndl) <~ 4
        return ()
        
runState st (Moo (4,(5,6)))

(1,(2,5))

fromList [(1,'c'),(2,'b')]

(1,Foo {_n = 2})

(1,Bar {_foo = Foo {_n = 2}})

(7,Foo {_n = 10})

((),Moo {_t = (1,(2,4))})